In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
# load data from CSV files
txn_df = pd.read_csv('transactions_20240325.csv')

/tmp/ipykernel_29022/1847196233.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  txn_df = pd.read_csv('transactions_20240325.csv')


In [36]:
txn_df.head()

,id,trx_id,trx_type,customer_id,device_serial,device_model,trx_date,trx_reference,credit_amount,debit_amount,balance_amount,status,status_message
0,52824,baed3442-519d-11ec-a479-022ca9a9afd4,ADD_MONEY,169,NaN,NaN,2021-11-30 05:24:00,4953063803,200,0,2000,PROCESSED,OK
1,52829,788ffd63-51a2-11ec-a479-022ca9a9afd4,DEVICE_RECHARGE,143,0000316,GTKW35TV16WZ,2021-11-30 05:57:00,1day plan,0,2800,2400,PROCESSED,OK
2,52834,1ae760ec-51b5-11ec-a479-022ca9a9afd4,ADD_MONEY,126,NaN,NaN,2021-11-30 08:11:00,3284053450,200,0,300,PROCESSED,OK
3,52839,97516416-51b6-11ec-a479-022ca9a9afd4,ADD_MONEY,145,NaN,NaN,2021-11-30 08:22:00,2609216024,3000,0,4200,PROCESSED,OK
4,52843,feba84d0-51b8-11ec-a479-022ca9a9afd4,DEVICE_RECHARGE,23,0000120,GTKW50TV16VZ,2021-11-30 08:39:00,7days plan,0,14000,0,PROCESSED,OK


In [37]:
# cleaning transactions data.
#
# Remove Txn IDs since they do not contain any information
# Remove credit amount since will be interested in recharge transactions (actual purchases).
# Remove `status_message` since it is a duplicate of `status` column
# Convert dates from str to datetime objects
txn_df = txn_df[txn_df['trx_type'] == 'DEVICE_RECHARGE']
txn_df.drop(columns=['trx_id', 'id', 'credit_amount', 'status_message'], inplace=True)
txn_df['trx_date'] = pd.to_datetime(txn_df['trx_date'], format='ISO8601')

In [38]:
txn_df = txn_df[txn_df['status'] == 'PROCESSED']
txn_df.drop(columns=['status'], inplace=True)

In [46]:
txn_df.shape

(66806, 8)

In [66]:
txn_df.head()

,trx_type,customer_id,device_serial,device_model,trx_date,trx_reference,debit_amount,balance_amount,is_long_term_plan
1,DEVICE_RECHARGE,143,0000316,GTKW35TV16WZ,2021-11-30 05:57:00,1d,2800,2400,False
4,DEVICE_RECHARGE,23,0000120,GTKW50TV16VZ,2021-11-30 08:39:00,7d,14000,0,False
11,DEVICE_RECHARGE,158,0000174,GTKW60TV16UZ,2021-11-30 12:22:00,7d,16000,2300,False
19,DEVICE_RECHARGE,143,0000316,GTKW35TV16WZ,2021-12-02 21:30:00,1d,2800,3800,False
27,DEVICE_RECHARGE,28,0000102,GTKW50TV16VZ,2021-12-05 10:43:00,1d,3400,6800,False


### Frequency of transactions per customer

It is necessary to consider tenature in order to obtain necessecity for individual customers.

In [41]:
txns_per_customer = txn_df.groupby('customer_id').size().reset_index()

### Use `trx_reference` for categorizing plans

There might might be mispells in this column. Let us check how many categories do we have

In [63]:
txn_df['trx_reference'].unique()

array(['1d', '7d', '30d'], dtype=object)

In [44]:
# There a high possibility that more than 30 days plan indicates instalment customers.
# to be on a safer side, we're going to ignore those items

In [45]:
txn_df = txn_df[~txn_df['trx_reference'].isin(['1 Year', '0 Year', '2 years'])]

In [62]:
# We also need to rename the plans accordingly in order to improve sanity of our dataset
txn_df['trx_reference'].replace({
    "1day plan": "1d",
    "7days plan": "7d",
    "30days plan": "30d",
    "1 Day plan": "1d",
    "30 Days plan": "30d",
    "7 Days plan": "7d",
    "7 Days\n": "7d",
    "7 Days plan\n": "7d",
    "30 Days plan\n": "30d"
}, inplace=True);

In [65]:
txn_df['is_long_term_plan'] = txn_df['trx_reference'].apply(lambda x: x == '30d')

In [67]:
long_term_plans_per_customer = txn_df.groupby('customer_id')['is_long_term_plan'].sum().reset_index()

#### Feature engineering for financial stability
Calcualte average transaction amount per customer

In [69]:
avg_txn = txn_df.groupby('customer_id').agg({'debit_amount': 'mean'})
avg_txn.rename(columns={'debit_amount': 'average_credit_amount'}, inplace=True)

In [73]:
# Merge features into a single dataframe
features_df = pd.merge(txns_per_customer, long_term_plans_per_customer, on='customer_id')

In [77]:
features_df = pd.merge(features_df, avg_txn, on='customer_id')

In [82]:
features_df.rename(columns={0: 'txn_freq', 'is_long_term_plan': '30d_plans', 'average_credit_amount': 'avg_recharges'}, inplace=True)

In [86]:
features_df

,customer_id,txn_freq,30d_plans,avg_recharges
0,22,-0.326757,-0.551904,-1.061997
1,23,3.801635,-0.497846,-1.064850
2,25,0.203415,0.042734,-0.643168
3,27,0.142576,-0.551904,-1.112921
4,28,5.748499,-0.443788,-1.243908
...,...,...,...,...
1155,1609,-0.491893,-0.551904,-1.229504
1156,1619,-0.483202,-0.497846,-0.060787
1157,1621,-0.483202,-0.497846,0.524940
1158,1626,-0.491893,-0.551904,-1.229504


#### Scaling features

In [84]:
scaler = StandardScaler()

In [85]:
features_df[['txn_freq', '30d_plans', 'avg_recharges']] = scaler.fit_transform(features_df[['txn_freq', '30d_plans', 'avg_recharges']])

#### Modeling Necessity with KMeans Clustering

In [87]:
kmeans = KMeans(n_clusters=3)

In [88]:
features_df['necessity_cluster'] = kmeans.fit_predict(features_df[['txn_freq', '30d_plans']])

/home/moses/miniforge3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [89]:
features_df

,customer_id,txn_freq,30d_plans,avg_recharges,necessity_cluster
0,22,-0.326757,-0.551904,-1.061997,0
1,23,3.801635,-0.497846,-1.064850,2
2,25,0.203415,0.042734,-0.643168,0
3,27,0.142576,-0.551904,-1.112921,0
4,28,5.748499,-0.443788,-1.243908,2
...,...,...,...,...,...
1155,1609,-0.491893,-0.551904,-1.229504,0
1156,1619,-0.483202,-0.497846,-0.060787,0
1157,1621,-0.483202,-0.497846,0.524940,0
1158,1626,-0.491893,-0.551904,-1.229504,0


In [90]:
features_df['necessity_cluster'].unique()

array([0, 2, 1], dtype=int32)

In [109]:
features_df[features_df['necessity_cluster'] == 0]

,customer_id,txn_freq,30d_plans,avg_recharges,necessity_cluster
0,22,-0.326757,-0.551904,-1.061997,0
2,25,0.203415,0.042734,-0.643168,0
3,27,0.142576,-0.551904,-1.112921,0
5,31,0.064353,0.745488,0.499633,0
6,32,-0.274609,-0.335672,-0.577877,0
...,...,...,...,...,...
1155,1609,-0.491893,-0.551904,-1.229504,0
1156,1619,-0.483202,-0.497846,-0.060787,0
1157,1621,-0.483202,-0.497846,0.524940,0
1158,1626,-0.491893,-0.551904,-1.229504,0


#### Modeling Fiancial stability with Random Forest Classifier

In [107]:
features_df[features_df['customer_id'].isin([1402])]

,customer_id,txn_freq,30d_plans,avg_recharges,necessity_cluster
1098,1402,-0.47451,-0.443788,0.354331,0
